In [104]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# collaborative filtering (CF)

> cf 基于 一些模式（评分， 购买行为）， 不需要用户和商品 外生的信息， 实现商品推荐的目的。

推荐系统依赖输入的类型。
* explicit feedback 显式反馈 --> 电影评分， 点赞 ，订阅
* implicit feedback 隐式反馈 --> 购买历史，浏览历史，搜索历史，甚至鼠标点击。

#### CF系统的建立，需要两个实体--> items and users

In [105]:
# 读取数据 100000 行 user -item -rating 的数据
header = ['user_id','item_id','rating','timestamp']
df = pd.read_csv("ml-100k/u.data",sep = '\t',names = header)
df.iloc[:10]

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


#### CF系统主要分为二种实现技术
* Neighborhood approach (比较 users 和 items 之间的关系)
    * user-based CF
    * item-based CF
* latent factor models (user,item 转化到对应的隐因子空间)
    * matrix factorization 矩阵分解 (试图从用户的评分中**自动**的推测出user 和item的内在属性)

# Baseline predictors 基准线 
>协同过滤模型（users-items matrix） 试图从 用户的评分中获取 user 和 item 之间的交互信息（interactions。但是有些信息会独立于交互而存在: user bias/ item bias 用户/物品的偏差
* user bias --> 比如 某些用户喜欢打高分
* item bias --> 比如 有些物品评分高于其他物品

所以 既要考虑到 用户于产品的交互信息(user-item interactions)， 又要封装用户和产品的偏差影响。
#### baseline predictors:
$$\quad b_{ui} = \mu + b_u + b_i $$


In [106]:
# Mu: overall average rating 
Mu = np.average(df['rating'])
# observed deviations of user
bias_u= 0
# observed deviations of item
bias_i =0

比如： joe评价泰坦尼克号这部电影，网站电影均分$\mu$为3.7，泰坦尼克号高于平均，$b_i$ 为0.5，joe 是一个苛刻的用户，打分低于平均，$b_u$ 为-0.3. 所以 最终的$b_{ui}$ 为3.7 + 0.5 - 0.3 = 3.9

#### 估计 $b_i$ 和 $b_u$   使用最小二乘法
\begin{equation}
\min_{b_i,b_u}\sum_{u,i}(r_{ui} - \mu - b_i -b_u)^2  + \lambda(\sum_u b_u^2 + \sum_i b_i^2)
\end{equation}

给定 一定的评分数据$r_{ui}$， 使用**SGD随机梯度下降法**来估计参数 




#### 另一种准确度相对低的方法
对某个物品求该物品偏差
\begin{equation}
b_i = \frac{\sum_{u \in R(i)} (r_{ui} - \mu)}{\lambda_2 + |R(i)|}
\end{equation}

In [107]:
# 计算 ID为242 的电影 偏差
item_id=242
def calculate_bias_i(item_id):
    lambda2 = 25
    r_ui= df[df['item_id'] == item_id]
    b_i = np.sum([r - Mu for r in r_ui["rating"]])/(lambda2 + len(r_ui))
    return b_i
print("电影242的偏差为: {}".format(calculate_bias_i(242)))

电影242的偏差为: 0.3803266197183098


对某用户计算其打分偏好
\begin{equation}
b_u = \frac{\sum_{i \in R(u)} (r_{ui} - \mu - b_i)}{\lambda_3 + |R(u)|}
\end{equation}

In [108]:
# 计算所有电影的偏差
bias_i ={}
item_set = set(df['item_id'])
for item_id in item_set:
    bias_i[item_id] = calculate_bias_i(item_id)
# 计算 ID为22 的用户打分偏差
user_id = 22
def calculate_bias_u(user_id):
    lambda3 = 10
    r_ui= df[df['user_id'] == user_id]
    b_u= np.sum([e[1]['rating'] - Mu - bias_i[e[1]['item_id']] for e in list(r_ui[['item_id','rating']].iterrows())])/(lambda3 + len(r_ui))
    return b_u
       
print("用户22的打分偏差为: {}".format(calculate_bias_u(user_id)))

用户22的打分偏差为: -0.07795944320059058


$\lambda_2$ 和 $\lambda_3$ 为正则化系数， 一般有交叉验证(corss validation)求得. 这里先用Netflix数据集的参数。

# 评估方法 RMSE
评估 预测值 $\hat{r_{ui}}$ 的质量， 我们选用 root mean squred error 
\begin{equation}
\sqrt{\frac{\sum_{u,i \in Testset}(r_{ui} - \hat{r_{ui}})^2}{|TestSet|}}
\end{equation}

# Matrix factorization models
## SVD
* examples include pLSA , neural networks, Latent Dirichlet Allocation(LDA)
* model : SVD-based model --> 对 user-item ratings 矩阵分解 / user,item 转化到对应的隐因子空间
    * user $u$ --> vector $p_u$
    * item $i$ --> vector $q_i$
    * result -->  dot product  $q_i^T p_u$ 来获取 user-item interactions

加上 baseline predictors 最终的 预测值为
\begin{equation}
\hat{r_{ui}} = \mu + b_i + b_u + q_i^T p_u
\end{equation}

#### loss function
\begin{equation}
\min_{b*,q*,p*}\sum_{u,i}(r_{ui} - \mu - b_i -b_u - q_i^T p_i)^2  + \lambda_4( b_u^2 +  b_i^2 + \lVert q_i\rVert^2 + \lVert p_u\rVert^2)
\end{equation}

$\lambda_4$ 是正则化系数， 通常由cross validation决定.  Netflix data 数据为0.02.
####  optimazation 
一种简单的办法是 随机梯度下降SGD。 对给定的数据， 不断朝参数梯度的反方向更新.
\begin{equation}
e_{ui} = r_{ui} - \mu - b_i - b_u -q_i^T p_u \\
b_u \gets b_u + \gamma(e_{ui} - \lambda_4 b_u) \\
b_i \gets b_i + \gamma(e_{ui} - \lambda_4 b_i) \\
p_u \gets p_u + \gamma(e_{ui} q_i - \lambda_4 p_u) \\
q_i \gets q_i + \gamma(e_{ui} p_u - \lambda_4 q_i)
\end{equation}

$\gamma$ 为学习率， Netflix data 的参数为 0.005.

## SVD++
在SVD中只考虑了rating一项显式反馈。我们知道考虑隐式反馈会提高预测的准确度。在SVD模型中，一个可以抓住关键信息的隐式反馈就是对该电影$q_i$评过分的用户$p_u$自身评分的所有电影集合的信息$R(u)$, 这个信息反映了该用户自身的用户偏好，可以和$P_u$向量结合使用。在模型中，我们选用         $|R(u)|^{-\frac{1}{2}}\sum_{j \in R(u)} y_i$ 来表示隐式反馈

#### loss function
\begin{equation}
\hat{r_{ui}} = \mu + b_i + b_u + q_i^T (p_u + |R(u)|^{-\frac{1}{2}}\sum_{j \in R(u)} y_i) \\ 
\min_{b*,q*,p*}\sum_{u,i}(r_{ui} - \hat{r_{ui}})^2  + \lambda_5( b_u^2 +  b_i^2 + \lVert q_i\rVert^2 + \lVert p_u\rVert^2 + \sum_{j \in R(u)}y_i^2)
\end{equation}

#### optimization 优化算法同样选用随机梯度下降法
\begin{equation}
e_{ui} = r_{ui} - \hat{r_{ui}} \\
b_u \gets b_u + \gamma(e_{ui} - \lambda_5 b_u) \\
b_i \gets b_i + \gamma(e_{ui} - \lambda_5 b_i) \\
p_u \gets p_u + \gamma(e_{ui} q_i - \lambda_6 p_u) \\
q_i \gets q_i + \gamma(e_{ui} \bullet \big( p_u + |R(u)|^{-\frac{1}{2}}\sum_{j \in R(u)} y_i \big) - \lambda_6 q_i) \\
y_i \gets y_i + \gamma(e_{ui}||R(u)|^{-\frac{1}{2}} q_i - \lambda_6 y_i)
\end{equation}
$\gamma$ 为0.007，$\lambda_5$ 为0.005，$\lambda_6$ 为 0.015

